## 기본 import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
# 한글 출력을 위한 설정
from matplotlib import font_manager, rc
import platform
if platform.system()=='Darwin':
    rc('font', family='AppleGothic')
elif platform.system()=='Windows':
    font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font', family=font_name)
# 음수 사용(마이너스 기호 깨짐 방지)
plt.rcParams['axes.unicode_minus'] = False

## data 준비

In [3]:
df=pd.read_csv('./data/stock-data.csv')
print(df.head())
df.info()
# date는 날짜 데이터 같은데 object type으로 되어있다.

         Date  Close  Start   High    Low  Volume
0  2018-07-02  10100  10850  10900  10000  137977
1  2018-06-29  10700  10550  10900   9990  170253
2  2018-06-28  10400  10900  10950  10150  155769
3  2018-06-27  10900  10800  11050  10500  133548
4  2018-06-26  10800  10900  11000  10700   63039
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB


In [10]:
# 날짜 자료형을 변경해서 새로운 필드로 저장하기
df['NewDate']=pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20 entries, 2018-07-02 to 2018-06-01
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     20 non-null     object        
 1   Close    20 non-null     int64         
 2   Start    20 non-null     int64         
 3   High     20 non-null     int64         
 4   Low      20 non-null     int64         
 5   Volume   20 non-null     int64         
 6   NewDate  20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 1.8+ KB


In [11]:
# 새로 만들어진 날짜 컬럼을 인덱스로 지정하고 기존의 날짜 컬럼 삭제
df.set_index(df['NewDate'], inplace=True)
df.drop('Date',axis=1, inplace=True)
df.head()

,Close,Start,High,Low,Volume,NewDate
NewDate,,,,,,
2018-07-02,10100,10850,10900,10000,137977,2018-07-02
2018-06-29,10700,10550,10900,9990,170253,2018-06-29
2018-06-28,10400,10900,10950,10150,155769,2018-06-28
2018-06-27,10900,10800,11050,10500,133548,2018-06-27
2018-06-26,10800,10900,11000,10700,63039,2018-06-26


### 샘플 데이터 추가하기

In [15]:
# 첫 번째 데이터는 날짜로 변경 가능한 데이터
# 두 번째 데이터는 날짜로 변경 불가능한 데이터 (TM 오타)
date_strings=np.array([
    '03-04-2005 11:35 PM', '04-09-2005 09:09 TM'])
# 예외 발생
# print([pd.to_datetime(date, format='%d-%m-%Y %I:%M %p') for date in date_strings])
# errors 관련 (예외) 옵션이 존재한다.
print([pd.to_datetime(date, format='%d-%m-%Y %I:%M %p', errors='ignore') for date in date_strings])
# list에서는 데이터의 자료형이 달라도 상관없지만,
# array나 DF 로 변환해서 사용할 때 문제가 발생할 수 있기에 권장하지 않습니다.
print([pd.to_datetime(date, format='%d-%m-%Y %I:%M %p', errors='coerce') for date in date_strings])
# 변환이 되지 않는 경우에는 coerce는 NaT로 변환해버림
# 이러면 array나 df 변환 시 datatype은 보존할 수 있다.
print([pd.to_datetime(date, format='%d-%m-%Y %I:%M %p', errors='raise') for date in date_strings])
# errors='raise'는 default로 처음과 같이
# 에러가 발생하면 예외발생처리하고 종료해버린다.

[Timestamp('2005-04-03 23:35:00'), '04-09-2005 09:09 TM']
[Timestamp('2005-04-03 23:35:00'), NaT]


In [20]:
date_strings=np.array([
    '2023-01-01', '2023-02-02','2023-02-20', '2023-04-05'])
pddates=pd.to_datetime(date_strings)
pr_months=pddates.to_period(freq='M')
pr_months 
# 이렇게 된다면 '일' 이 잘리고 '월' 까지만 남는다.

PeriodIndex(['2023-01', '2023-02', '2023-02', '2023-04'], dtype='period[M]')

In [24]:
# 2023년 1월 1일부터 월 단위로 12개를 생성해보고 싶어
ts_ms=pd.date_range(start='2023-01-01',
                    periods=12,
                    freq='MS', # 월초
                    tz='Asia/Seoul')
print(ts_ms)

DatetimeIndex(['2023-01-01 00:00:00+09:00', '2023-02-01 00:00:00+09:00',
               '2023-03-01 00:00:00+09:00', '2023-04-01 00:00:00+09:00',
               '2023-05-01 00:00:00+09:00', '2023-06-01 00:00:00+09:00',
               '2023-07-01 00:00:00+09:00', '2023-08-01 00:00:00+09:00',
               '2023-09-01 00:00:00+09:00', '2023-10-01 00:00:00+09:00',
               '2023-11-01 00:00:00+09:00', '2023-12-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='MS')


In [31]:
# 데이터를 가져와서 문자열을 날짜로 다시 변경해두자.
df=pd.read_csv('./data/stock-data.csv')
df['NewDate']=pd.to_datetime(df['Date'])
df.set_index('NewDate', inplace=True)
df.drop('Date',axis=1, inplace=True)
# df.head()
# 날짜 인덱스여서 일부분만 가지고도 인덱싱 가능합니다.
# 문자열은 죽었다 깨어나도 안됩니다.
# df_y=df['2018-06']
# print(df_y.head())
# 범위도 가능하다, 어떤 컬럼을 할건지까지 할거면 loc 해주자.
df_y=df.loc['2018-06-25':'2018-06-30', 'Start':'Low']
print(df_y.head())

            Close  Start   High    Low  Volume
NewDate                                       
2018-06-29  10700  10550  10900   9990  170253
2018-06-28  10400  10900  10950  10150  155769
2018-06-27  10900  10800  11050  10500  133548
2018-06-26  10800  10900  11000  10700   63039
2018-06-25  11150  11400  11450  11000   55519


In [40]:
# 결측치 처리
time_index=pd.date_range('01-01-2023', periods=5, freq='M')
dataframe=pd.DataFrame(index=time_index)
dataframe['Sales']=[1.0,2.0,np.nan, np.nan, 5.0]
# print(dataframe)
# 앞의 데이터로 채우기
# print(datafram.ffill())
# 선형 보간 - 일차식
print(dataframe.interpolate())
# 비선형 보간 - 다항식 
# print(dataframe.interpolate(method='quadratic'))

            Sales
2023-01-31    1.0
2023-02-28    2.0
2023-03-31    3.0
2023-04-30    4.0
2023-05-31    5.0


In [43]:
dataframe['Stock_Price']=[1,2,3,4,5]
# 단순 이동 평균
print(dataframe.rolling(window=2).mean())
# 지수 이동 평균
print(dataframe.ewm(span=2).mean())

            Sales  Stock_Price
2023-01-31    NaN          NaN
2023-02-28    1.5          1.5
2023-03-31    NaN          2.5
2023-04-30    NaN          3.5
2023-05-31    NaN          4.5
               Sales  Stock_Price
2023-01-31  1.000000     1.000000
2023-02-28  1.750000     1.750000
2023-03-31  1.750000     2.615385
2023-04-30  1.750000     3.550000
2023-05-31  4.847059     4.520661
